# Recommendation Engine

In [5]:
import numpy as np
import pandas as pd

In [6]:
# load data
df_movie = pd.read_csv("data/movies_metadata.csv")
df_ratings = pd.read_csv("data/ratings_small.csv")
pd.set_option('display.max_columns', None)
df_movie.head()

/var/folders/5_/39stv43d5_70psn53dfmjwt00000gn/T/ipykernel_11545/2682971138.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movie = pd.read_csv("data/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
df_movie['movieId'] = pd.to_numeric(df_movie['id'], errors='coerce')
df = pd.merge(df_ratings, df_movie, on='movieId', how='inner')


In [8]:
df_re = df.iloc[:, 0:3]
df_re

,userId,movieId,rating
0,1,1371,2.5
1,4,1371,4.0
2,7,1371,3.0
3,19,1371,4.0
4,21,1371,3.0
...,...,...,...
44989,652,129009,4.0
44990,653,2103,3.0
44991,659,167,4.0
44992,659,563,3.0


## EDA

## Recommendation Engine

In [10]:
user_item = pd.pivot_table(data=df_re, values='rating', index='userId', columns='movieId')
user_item.replace('',np.nan,inplace=True)
user_item.head()



movieId  2       3       5       6       11      12      13      14      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     4.0     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  15      16      17      18      19      20      21      22      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     5.0     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  24      25      26      27      28      30      35      38      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  55      58      59      62      63      64      65      66      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     3.0     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  68      69      70      71      73      74      76      77      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  78      79      80      81      82      83      85      86      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  87      88      89      92      93      95      96      97      \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
5           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

movieId  98      99      100     101     102     103     104     105     \
userId                                                                    
1           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2           NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
3           NaN   

In [14]:
# Normalize user-item matrix
user_item_norm = user_item.subtract(user_item.mean(axis=1), axis = 0)
user_item_norm.head()
M = user_item_norm.fillna(0)
M

movieId  2         3       5        6        11      12      13      14      \
userId                                                                        
1           0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   
2           0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   
3           0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   
4           0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   
5           0.0  0.084906     0.0  0.00000  0.00000     0.0     0.0     0.0   
...         ...       ...     ...      ...      ...     ...     ...     ...   
667         0.0  0.000000     0.0  0.45614 -0.54386     0.0     0.0     0.0   
668         0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   
669         0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   
670         0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   
671         0.0  0.000000     0.0  0.00000  0.00000     0.0     0.0     0.0   

movieId  15       16        17      18      19      20       21      22      \
userId                                                                        
1           0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   
2           0.0  0.00000  1.482759     0.0     0.0     0.0  0.00000     0.0   
3           0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   
4           0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   
5           0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   
...         ...      ...       ...     ...     ...     ...      ...     ...   
667         0.0 -1.54386 -1.543860     0.0     0.0     0.0 -0.54386     0.0   
668         0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   
669         0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   
670         0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   
671         0.0  0.00000  0.000000     0.0     0.0     0.0  0.00000     0.0   

movieId  24        25      26      27      28      30      35      38      \
userId                                                                      
1           0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   
2           0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   
3           0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   
4           0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   
5           0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   
...         ...       ...     ...     ...     ...     ...     ...     ...   
667         0.0 -1.543860     0.0     0.0     0.0     0.0     0.0     0.0   
668         0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   
669         0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   
670         0.0  1.235294     0.0     0.0     0.0     0.0     0.0     0.0   
671         0.0  0.000000     0.0     0.0     0.0     0.0     0.0     0.0   

movieId  55       58      59        62      63      64      65      66      \
userId                                                                       
1           0.0  0.00000     0.0  0.000000     0.0     0.0     0.0     0.0   
2           0.0  0.00000     0.0 -0.517241     0.0     0.0     0.0     0.0   
3           0.0  0.00000     0.0  0.000000     0.0     0.0     0.0     0.0   
4           0.0  0.00000     0.0  0.000000     0.0     0.0     0.0     0.0   
5           0.0  0.00000     0.0  0.000000     0.0     0.0     0.0     0.0   
...         ...      ...     ...       ...     ...     ...     ...     ...   
667         0.0  0.45614     0.0  0.000000     0.0     0.0     0.0     0.0   
668         0.0  0.00000     0.0  0.000000     0.0     0.0     0.0     0.0   
669         0.0  0.00000     0.0  0.000000     0.0     0.0     0.0     0.0   
670         0.0  0.00000     0.0  0.000000     0.0     0.0     0.0     0.0   
671   

In [24]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances

M = pd.DataFrame(M)

#declaring k,metric as global which can be changed by the user later
global k, metric
k = 4
metric = 'cosine'  #can be changed to 'correlation' for Pearson correlation similaries
M

#get cosine similarities for ratings matrix M; pairwise_distances returns the distances between ratings and hence
#similarities are obtained by subtracting distances from 1
cosine_sim = 1 - pairwise_distances(M, metric="cosine")
#Cosine similarity matrix
pd.DataFrame(cosine_sim)
#get pearson similarities for ratings matrix M
pearson_sim = 1 - pairwise_distances(M, metric="correlation")
#Pearson correlation similarity matrix
pd.DataFrame(pearson_sim)


#This function finds k similar users given the user_id and ratings matrix M
#Note that the similarities are same as obtained via using pairwise_distances
def findksimilarusers(user_id, ratings, metric=metric, k=k):
    similarities = []
    indices = []
    model_knn = NearestNeighbors(metric=metric, algorithm='brute')
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[user_id - 1, :].values.reshape(1, -1), n_neighbors=k + 1)
    similarities = 1 - distances.flatten()
    print('{0} most similar users for User {1}:\n'.format(k, user_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] + 1 == user_id:
            continue;

        else:
            print(
                '{0}: User {1}, with similarity of {2}'.format(i, indices.flatten()[i] + 1, similarities.flatten()[i]))

    return similarities, indices


similarities, indices = findksimilarusers(1, M, metric='cosine')
#similarities, indices = findksimilarusers(1, M, metric='correlation')



4 most similar users for User 1:

1: User 420, with similarity of 0.22512784348366655
2: User 87, with similarity of 0.19588501215712295
3: User 41, with similarity of 0.17008845402261297
4: User 606, with similarity of 0.12547525776326895


In [18]:

#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, ratings, metric = metric, k=k):
    prediction=0
    similarities, indices=findksimilarusers(user_id, ratings,metric, k) #similar users based on cosine similarity
    mean_rating = ratings.loc[user_id,:].mean()
    sum_wt = np.sum(similarities)-1
    product=1
    wtd_sum = 0

    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i]+1 == user_id:
            continue;
        else:
            ratings_diff = ratings.iloc[indices.flatten()[i],item_id-1]-np.mean(ratings.iloc[indices.flatten()[i],:])
            product = ratings_diff * (similarities[i])
            wtd_sum = wtd_sum + product

    prediction = int(round(mean_rating + (wtd_sum/sum_wt)))
    print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

predict_userbased(1, 100, M, k=3);


4 most similar users for User 1:

1: User 420, with similarity of 0.22512784348366655
2: User 87, with similarity of 0.19588501215712295
3: User 41, with similarity of 0.17008845402261297
4: User 606, with similarity of 0.12547525776326895

Predicted rating for user 1 -> item 100: 0


In [23]:
#This function finds k similar items given the item_id and ratings matrix M

def findksimilaritems(item_id, ratings, metric=metric, k=k):
    similarities = []
    indices = []
    ratings = ratings.T
    model_knn = NearestNeighbors(metric=metric, algorithm='brute')
    model_knn.fit(ratings)

    distances, indices = model_knn.kneighbors(ratings.iloc[item_id - 1, :].values.reshape(1, -1), n_neighbors=k + 1)
    similarities = 1 - distances.flatten()
    print('{0} most similar items for movie {1}:\n'.format(k, item_id))
    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] + 1 == item_id:
            continue;

        else:
            print('{0}: Movie {1} :, with similarity of {2}'.format(i, indices.flatten()[i] + 1,
                                                                   similarities.flatten()[i]))

    return similarities, indices


similarities, indices = findksimilaritems(3, M)

#This function predicts the rating for specified user-item combination based on item-based approach
def predict_itembased(user_id, item_id, ratings, metric=metric, k=k):
    prediction = wtd_sum = 0
    similarities, indices = findksimilaritems(item_id, ratings)  #similar users based on correlation coefficients
    sum_wt = np.sum(similarities) - 1
    product = 1

    for i in range(0, len(indices.flatten())):
        if indices.flatten()[i] + 1 == item_id:
            continue;
        else:
            product = ratings.iloc[user_id - 1, indices.flatten()[i]] * (similarities[i])
            wtd_sum = wtd_sum + product
    prediction = int(round(wtd_sum / sum_wt))
    print('\nPredicted rating for user {0} -> Movie {1}: {2}'.format(user_id, item_id, prediction))

    return prediction
prediction = predict_itembased(1, 3, M, k= 3)


4 most similar items for movie 3:

1: Movie 24 :, with similarity of 0.3575070575247884
2: Movie 316 :, with similarity of 0.3420045289017588
3: Movie 86 :, with similarity of 0.32305814796639476
4: Movie 368 :, with similarity of 0.3033734155921495
4 most similar items for movie 3:

1: Movie 24 :, with similarity of 0.3575070575247884
2: Movie 316 :, with similarity of 0.3420045289017588
3: Movie 86 :, with similarity of 0.32305814796639476
4: Movie 368 :, with similarity of 0.3033734155921495

Predicted rating for user 1 -> Movie 3: 0


In [22]:
prediction

0